# CS-340 Module 7 Project 2
## Christopher Clark
## August 14, 2022

### Included new funtionality for a dropdown menu with the four selections provided in the assignment. The 
### reset feature returns all results when selected. A pie chart has been added to display graph data based 
### on the dropdown menu item selected. The geochart updates the location for the first animal in the list 
### when different selections are made. 
### The embedded link to snhu.edu is incorporated into the Grazioso Salvare logo at the top of the page.
### It is best to right click and open the link in a new window as there is no back button to return to the 
### dashboard.

In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name

#import sys and all finctionality
import sys
#set path to module
sys.path.append("/home/christophercl1_snhu/Desktop")
#import AnimalShelter class from crud module
from crudbackup import AnimalShelter


###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name

#hardcoded username and password
username = "aacuser"
password = "1234"
#instantiate the animal shelter class and pass the username and password
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('CS-340 Project Two')

#FIX ME Add in Grazioso Salvare’s logo

#Add and encode image file provided in assignment
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    #html.Div(id='hidden-div', style={'display':'none'}),
    
    #Code added for link to snhu.edu embedded in the Grazioso Salvare
    #logo. This is a one-time click. There is no back button to return
    #to the dashboard. Open the link in a new tab or restart the prorgram to access dashboard after
    #clicking on logo linked to snhu.edu
    html.Div(        
    html.A([html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                    style={'height': 200,
                          'width': 200,
                         },
                    alt="Grazioso Salvare")],
                    href='https://www.snhu.edu')
    ),
    
    #Dashboard header with unique identifier. Unique identifier has style 
    #options added for font size and color.
    html.Center(html.B(html.H1('Grazioso Salvare Database Dashboard'))),
    html.Center(html.B('Website Design by Christopher Clark', style={'color':'red','fontSize': 12,})),    
    html.Hr(),
    html.Div(
        
        #FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
        
        #Code to build a dropdown menu for filtering options. The four widgets required
        #in the assignment have been added. 
        dcc.Dropdown(
        id='dropdown-menu',
        options=[
            {'label': 'Water Rescue', 'value': 'Water Rescue'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'Wilderness Rescue'},
            {'label': 'Disaster Rescue or Individual Tracking', 'value': 'Diaster Rescue'},
            {'label': 'Return All Results', 'value': 'All Records'}
        ],
           
        clearable=False,
        placeholder="Make Selection"
    )

    ),
    #Datable set up with column options
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        style_header = {
            'padding': '20px', 'fontWeight': 'bold'
        },
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        #If you completed the Module Six Assignment, you can copy in the code you created here 
        
        #Dictionary options 
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable = "True",
        row_selectable='single',
        selected_rows=[],
        selected_columns=[],
        page_current=0,
        page_size=20,
    ),
        
    html.Br(),
    html.Hr(),
    
    #Geolocation and piechart set up to display below dashboard and 
    #side-by-side to each other.
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',
            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
        )
    ])
])



#############################################
# Interaction Between Components / Controller
#############################################

#Callback for dropdown menu
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('dropdown-menu', 'value')])
def dashboardFilterResult(value):
### FIX ME Add code to filter interactive data table with MongoDB queries  

    #Define method to filter dashboard reults based on the four filtering options
    #in the dropdown menu. When selected the filter options will query the database
    #as shown here and return those results to the dashboard. When 'return all results' is 
    #selected, the dashboard returns all reults in the dict.
    if value == 'Water Rescue':
        df = pd.DataFrame.from_records(shelter.read(
            {"animal_type":"Dog",
             "breed":{"$in":["Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"]},
             "sex_upon_outcome":"Intact Female",
             "age_upon_outcome_in_weeks":{"$gte":26},"age_upon_outcome_in_weeks":{"$lte":156}}))
    elif value == 'Wilderness Rescue':
        df = pd.DataFrame.from_records(shelter.read(
            {"animal_type":"Dog",
             "breed":{"$in":["German Shepherd","Alaskan Malamute","Old English Sheepdog","Siberian Husky","Rottweiler"]},
             "sex_upon_outcome":"Intact Male",
             "age_upon_outcome_in_weeks":{"$gte":26},"age_upon_outcome_in_weeks":{"$lte":156}}))
    elif value == 'Diaster Rescue':
        df = pd.DataFrame.from_records(shelter.read(
           {"animal_type":"Dog",
            "breed":{"$in":["Doberman Pinscher","German Shepherd","Golden Retriever","Bloodhound","Rottweiler"]},
            "sex_upon_outcome":"Intact Male",
            "age_upon_outcome_in_weeks":{"$gte":20},"age_upon_outcome_in_weeks":{"$lte":300}}))
    else:
        df = pd.DataFrame.from_records(shelter.read({}))
        
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
    
    
    return (data,columns)


#Callbac for the datatable and stuyle options to highlight a column when selected
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def highlightColumn(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


#Callback to build the pie chart
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "data")])


def updateGraphData(data):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    #return [
    #    dcc.Graph(            
    #        figure = ###
    #    )    
    #]
    
    #Define method to create and update the piechart data when different search 
    #item is selected fro the dropdown menu
    dff = pd.DataFrame.from_dict(data)
    fig=px.pie(dff,
                 names='breed',
                 title="Breeds Available by Rescue Type"
                 )
    fig.update_traces(textposition='inside')
    return [
        dcc.Graph(
            id='graph=id',
            figure=fig
        )
    ]

#Callback for geolocation chart
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
    Input('datatable-id', "selected_rows")])
def updateGeoChart(viewData, row):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
        
    #Code for method to updating the geochart display when
    #different filtering options selected in dropdown menu
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
            dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
                # Marker with tool tip and popup
                dl.Marker(position=[30.75,-97.748],
                          children=[
                              dl.Tooltip(dff.iloc[0,4]),
                              dl.Popup([
                                  html.H1("Animal Name"),
                                  html.P(dff.iloc[1,9]),
                                  
                              ])
                          ])
            ])
        ]
    
    


app